In [11]:
import pandas as pd
import numpy as np

### 1. Data Set DownLoad

In [12]:
kospi_data = pd.read_csv('./data/final_kospi.csv')
kosdaq_data = pd.read_csv('./data/final_kosdaq.csv')

c:\Users\bkjeo\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (19,21,23,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [25]:
processed_kospi_data = kospi_data.copy(deep=True)

unnecessary_columns = ['시가', '고가', '저가', '종가', '날짜', 'CODE', 'code_y']
processed_kospi_data = processed_kospi_data.copy().drop(columns=unnecessary_columns, inplace=False)
processed_kospi_data = processed_kospi_data.dropna(axis=0) # null 값 제가 
processed_kospi_data = processed_kospi_data.reset_index(drop=True)
strict=False

len(processed_kospi_data)

366904

### 2. Split Data Set
Split by 60% train data, 20% valid data, 20% test data

In [26]:
import sklearn
from sklearn.model_selection import train_test_split

x = processed_kospi_data.copy().drop(columns=['Y'], inplace=False)
y = processed_kospi_data['Y']

x_train, x_remain, y_train, y_remain = train_test_split(z, y, train_size = 0.8)
x_valid, x_test, y_valid, y_test = train_test_split(x_remain, y_remain, train_size=0.5)

x_train = x_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [30]:
print(X_train.columns)

Index(['Unnamed: 0', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS', 'code_x', '거래량',
       '시가총액', 'interest', '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계',
       '이익잉여금', '자본총계', '매출액', '영업이익', '법인세차감전 순이익', '당기순이익', '자본금'],
      dtype='object')


### 3. Data Normalization and Visualization

In [ ]:
## iqr 을 적용하여 outlier를 제외한 평균 분산 값으로 정규화

def calc_ (x) : 
    x_IQR = x.quantile(0.75) - x.quantile(0.25)
    lower_bound = (x > x.quantile(0.25) - x_IQR * 1.5) ## -2.7σ in Gaussian (if x ~ Gaussian)
    upper_bound = (x < x.quantile(0.75) + x_IQR * 1.5) ## 2.7σ in Gaussian
    bound_conditions = lower_bound & upper_bound
    x_clean = x.loc[bound_conditions]
    return (x_clean.mean(), x_clean.std())

def normalize(x_train, ) :
    x_IQR = x.quantile(0.75) - x.quantile(0.25)
    lower_bound = (x > x.quantile(0.25) - x_IQR * 1.5) ## -2.7σ in Gaussian (if x ~ Gaussian)
    upper_bound = (x < x.quantile(0.75) + x_IQR * 1.5) ## 2.7σ in Gaussian
    bound_conditions = lower_bound & upper_bound
    x_clean = x.loc[bound_conditions]
    return (x-x_clean.mean())/x_clean.std()

X_train